In [111]:
import pandas as pd
!ls -lh ../../../../00_data/STAD

total 12M
-rw-rw-r-- 1 shenwanxiang shenwanxiang 5.8K 3月  16 15:38 Kim_clinical.tsv
-rw-rw-r-- 1 shenwanxiang shenwanxiang  12M 3月  16 15:38 Kim_tpm.tsv


In [107]:
cohort = 'SU2CLC2'
dfc = pd.read_csv('../../../../00_data/LUSC/%s_clinical.tsv' % cohort,sep='\t', index_col=0)
dfc.to_csv('./%s_clinical.csv' % cohort)

dfc3 = pd.read_csv('../../../../00_data/LUSC/%s_tpm.tsv' % cohort, sep='\t', index_col=0)
dfc3.to_csv('./%s_tpm.csv' % cohort)

In [108]:
gene_lengths = pd.read_pickle('../gene_length//gene_length.pkl')

In [109]:
gene_lengths = gene_lengths[dfc3.columns]  # reorder to match dfc3

# 2. 转换基因长度为 kilobases (kb)
gene_lengths_kb = gene_lengths / 1000.0

# 3. 计算 scaling factor：每个样本的 TPM * gene_length 的总和
scaling_factors = (dfc3 * gene_lengths_kb).sum(axis=1)  # sum across genes per sample

# 4. TPM → counts
counts = dfc3.mul(gene_lengths_kb, axis=1)  # TPM * gene_length
counts = counts.mul(scaling_factors / 1e6, axis=0)  # * (scaling factor / 1e6) for each sample

# 5. 可选：四舍五入为整数
counts = counts.round().astype(int)
counts.to_csv('./%s_counts_impu.csv' % cohort)